In [4]:
import os
import numpy as np
import re
import pickle
import sys

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from ProcessEventsFunctions import *
# from Convert_to_Profiles_Functions import *
# from Get_Events_Functions import *

sys.path.insert(1, 'Old')
from Steef_Functions import *

In [5]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [6]:
quintile_mapping = {1: 'F2', 2: 'F1', 3: 'C', 4: 'B1', 5: 'B2'}
quintile_mapping_thirds = {1: 'F', 2: 'C', 3: 'B'}

In [7]:
tbo_vals = pd.read_csv(home_dir + 'datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# tbo_vals = tbo_vals[tbo_vals['Lon']!=-999.0]
# Check if the points are within the areas
tbo_vals = check_for_gauge_in_areas(tbo_vals, home_dir, ['NW', 'NE', 'ME', 'SE', 'SW'])
tbo_vals.loc[tbo_vals['within_area'] == 'NW, C', 'within_area'] = 'NW'
tbo_vals.loc[tbo_vals['within_area'] == 'ME, SE', 'within_area'] = 'ME'
tbo_vals['within_area'].unique()

array(['ME', 'SE', 'NE', 'NW', 'SW', ''], dtype=object)

### Get events (considering one set of AMAX producing events (with duplicates deleted)

In [7]:
# Now you can callevents_dict_present_nimrod, event_props_dict_present_nimrod, event_profiles_dict_present_nimrod = process_events_alltogether_nimrod(home_dir2, 'Present', tbo_vals)
events_dict_nimrod, event_props_dict_nimrod, event_profiles_dict_nimrod = process_events_alltogether_nimrod(home_dir2, tbo_vals)

Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_14256/3539429331.py", line 2, in <module>
    events_dict_nimrod, event_props_dict_nimrod, event_profiles_dict_nimrod = process_events_alltogether_nimrod(home_dir2, tbo_vals)
  File "/nfs/a319/gy17m2a/PhD/Scripts/FindIndependentRainfallEvents/ProcessEvents/ProcessEventsFunctions.py", line 257, in process_events_alltogether_nimrod
    this_event = read_event(gauge_num, fp)
  File "/nfs/a319/gy17m2a/PhD/Scripts/FindIndependentRainfallEvents/ProcessEvents/ProcessEventsFunctions.py", line 74, in read_event
    test = pd.read_csv(fp)
  File "/nfs/a321/gy17m2a/anaconda_install/anaconda3/envs/ukcp18/lib/python3.7/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/nfs/a321/gy17m2a/

TypeError: object of type 'NoneType' has no len()

In [16]:
# with open(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/events_dict_nimrod.pickle", 'wb') as handle:
#     pickle.dump(events_dict_nimrod, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/event_profiles_dict_nimrod", 'wb') as handle:
#     pickle.dump(event_profiles_dict_nimrod, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/event_props_dict_nimrod.pickle", 'wb') as handle:
#     pickle.dump(event_props_dict_nimrod, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Get events for each duration (there'll be cross over in events present for each duration)
By duration here we mean the duration for which the AMAX are associated, rather than the actual duration of the event

In [13]:
home_dir = home_dir2
tb0_vals = tbo_vals

events_dict = {duration: {} for duration in valid_durations}
event_props_dict = {duration: {} for duration in valid_durations}
event_profiles_dict = {duration: {} for duration in valid_durations}

for gauge_num in range(1007, 1008):
    if gauge_num not in [444, 827, 888]:
        print(gauge_num)
        if gauge_num % 100 == 0:
            print(f"Processing gauge {gauge_num}")

            indy_events_fp = home_dir + f"ProcessedData/IndependentEvents/NIMROD_30mins/NIMROD_2.2km_filtered_100/{gauge_num}/WholeYear/EventSet/"

            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)

            for event_num, file in enumerate(files):
                fp = indy_events_fp + f"{file}"
                if '2080' in fp:
                    continue

                # Get duration for this event
                duration_of_this_event = get_dur_for_which_this_is_amax(fp)
                if duration_of_this_event not in valid_durations:
                    continue  # Skip if not in the list of valid durations

                # print(f"Processing event file: {file} with duration: {duration_of_this_event}")

                # Get event
                this_event = read_event(gauge_num, fp)

                # Get times and precipitation values
                event_times = this_event['times']
                event_precip = this_event['precipitation (mm)']

                # Apply the function to adjust the dates in the 'times' column
                event_times_fixed = event_times.apply(adjust_feb_dates)

                # Create the DataFrame with corrected times
                event_df = pd.DataFrame({'precipitation (mm)': event_precip, 'times': event_times_fixed})

                # Create characteristics dictionary
                event_props = create_event_characteristics_dict(event_df)

                # Add the duration
                event_props['dur_for_which_this_is_amax'] = duration_of_this_event
                # Add gauge number and ensemble member
                event_props['gauge_num'] = gauge_num
                event_props['area'] = tb0_vals.iloc[gauge_num]['within_area']
                event_props['filename'] = file

                # Store results in corresponding duration category
                events_dict[f"nimrod, {gauge_num}, {event_num}"] = event_df
                event_props_dict[duration_of_this_event][f"nimrod, {gauge_num}, {event_num}"] = event_props
                event_profiles_dict[f"nimrod, {gauge_num}, {event_num}"] = create_profiles_dict(event_df)


1007


In [12]:
# List of desired durations
valid_durations = ["0.5", "1", "2", "3", "6", "12", "24"]

# Process events for both time periods
results_nimrod = process_events_by_duration_nimrod(home_dir2, valid_durations, tbo_vals)

Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
gauge 1007 encountered problem
Processing gauge 1100
Processing gauge 1200
gauge 1293 encountered problem


In [11]:
# List of desired durations
valid_durations = ["0.5", "1", "2", "3", "6", "12", "24"]

# Process events for both time periods
results_nimrod = process_events_by_duration_nimrod(home_dir2, valid_durations, tbo_vals)

Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
gauge 1007 encountered problem
Processing gauge 1100
Processing gauge 1200
gauge 1293 encountered problem


In [18]:
dur_nimrod_events_dict, dur_nimrod_event_props_dict, dur_nimrod_event_profiles_dict = results_nimrod

In [15]:
with open(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/results_each_dur_nimrod.pickle", 'wb') as handle:
    pickle.dump(results_nimrod, handle, protocol=pickle.HIGHEST_PROTOCOL)    

## Do we have the same number of results for each duration?
I believe that longer durations have more events because of compound annual maxima

In [19]:
for duration in valid_durations:
    print(duration, len(dur_present_event_props_dict[duration].keys()))

0.5 2964
1 2964
2 2964
3 2964
6 2966
12 3022
24 3449


In [33]:
# List of desired durations
duration_bins = ['<4hr', '4-12hr', '12hr+']

def process_events_by_duration(home_dir, time_period, duration_bins, ems, tb0_vals):
    events_dict = {duration: {} for duration in duration_bins}
    event_props_dict = {duration: {} for duration in duration_bins}
    event_profiles_dict = {duration: {} for duration in duration_bins}
    for em in ems:  # You can add more ensemble members as needed
        print(em)
        for gauge_num in range(0, 1293):
            if gauge_num not in [444, 827, 888]:
                if gauge_num % 100 == 0:
                    print(f"Processing gauge {gauge_num}")
                    indy_events_fp = home_dir + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet/"

                    files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
                    files = np.sort(files)

                    for event_num, file in enumerate(files):
                        fp = indy_events_fp + f"{file}"
                        if '2080' in fp:
                            continue

                        # Get event
                        this_event = read_event(gauge_num, fp)

                        # Get times and precipitation values
                        event_times = this_event['times']
                        event_precip = this_event['precipitation (mm)']

                        # Apply the function to adjust the dates in the 'times' column
                        event_times_fixed = event_times.apply(adjust_feb_dates)

                        # Create the DataFrame with corrected times
                        event_df = pd.DataFrame({'precipitation (mm)': event_precip, 'times': event_times_fixed})

                        # Create characteristics dictionary
                        event_props = create_event_characteristics_dict(event_df)
                        
                        # Add the duration
                        event_props['dur_for_which_this_is_amax'] = get_dur_for_which_this_is_amax(fp)
                        # Add gauge number and ensemble member
                        event_props['gauge_num'] = gauge_num
                        event_props['area'] = tb0_vals.iloc[gauge_num]['within_area']
                        event_props['em'] = em
                        event_props['filename'] = file

                        duration_category_of_this_event = event_props['DurationRange_simple']
                        # Store results in corresponding duration category
                        events_dict[duration_category_of_this_event][f"{em}, {gauge_num}, {event_num}"] = event_df
                        event_props_dict[duration_category_of_this_event][f"{em}, {gauge_num}, {event_num}"] = event_props
                        event_profiles_dict[duration_category_of_this_event][f"{em}, {gauge_num}, {event_num}"] = create_profiles_dict(event_df)

    return events_dict, event_props_dict, event_profiles_dict

# Process events for both time periods
results_present_dur_categories_simple = process_events_by_duration(home_dir2, 'Present', duration_bins, ems_present, tbo_vals)
results_future_dur_categories_simple = process_events_by_duration(home_dir2, 'Future', duration_bins, ems_future, tbo_vals)

bc005
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
Processing gauge 1100
Processing gauge 1200
bc006
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
Processing gauge 1100
Processing gauge 1200
bc007
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processing gauge 700
Processing gauge 800
Processing gauge 900
Processing gauge 1000
Processing gauge 1100
Processing gauge 1200
bc009
Processing gauge 0
Processing gauge 100
Processing gauge 200
Processing gauge 300
Processing gauge 400
Processing gauge 500
Processing gauge 600
Processin

In [36]:
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/results_present_dur_categories_simple.pickle", 'wb') as handle:
    pickle.dump(results_present_dur_categories_simple, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/results_future_dur_categories_simple.pickle", 'wb') as handle:
    pickle.dump(results_future_dur_categories_simple, handle, protocol=pickle.HIGHEST_PROTOCOL)    